# Face Recognition Model - Serving Function
The function uses PyTorch classifier on top of an opencv deep learning model to encode and recognize faces in given image

### install packages to deploy function manualy

In [98]:
#nuclio: ignore
from mlrun import load_project
from os import path

project_path = path.abspath('./')
project = load_project(project_path)

model_path = project.params.get('MODEL_PATH')
classes_map = project.params.get('CLASSES_MAP')
models_path = project.params.get('MODELS_PATH')
model_path = project.params.get('MODEL_PATH')
data_path = project.params.get('DATA_PATH')

print(classes_map)

avia/examples/faces/artifacts/idx2name.csv


In [99]:
#nuclio: ignore
import nuclio

### Install dependencies and set config

In [100]:
%%nuclio cmd -c
pip install opencv-python-headless
pip install imutils
pip install torch torchvision
pip install pandas
pip install v3io_frames
pip install scikit-build
pip install cmake==3.13.3
pip install face_recognition
pip install ipython

In [101]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Perform necessary imports

In [102]:
import importlib.util
import cv2
import face_recognition
import imutils
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import string
import v3io_frames as v3f
import os
import datetime
from pickle import load
import shutil
import base64
import IPython
from os import getenv

### Set function environment variables

In [103]:
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env MODEL_PATH=model_path
%nuclio env CLASSES_MAP=classes_map

%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'MODEL_PATH' environment variable
%nuclio: setting 'CLASSES_MAP' environment variable


### Model serving class

In [104]:
def jpg_str_to_frame(encoded):
    jpg_original = base64.b64decode(encoded)
    jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

class PytorchModel(object):
    def __init__(self):
        self.name = 'model.bst'
        self.model_filepath = os.environ['MODEL_PATH']
        self.model = None
        self.ready = None
        self.classes = os.environ['CLASSES_MAP']
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    def convert_classes_path_for_pandas(self):
        V3IO_USERNAME = getenv('V3IO_USERNAME')
        classes=self.classes
        classes = classes.replace(V3IO_USERNAME,'/User')
        return classes
    
    def load(self):
        input_dim = 128
        hidden_dim = 64
        output_dim = self.n_classes

        spec = importlib.util.spec_from_file_location('models', os.environ['MODELS_PATH'])
        models = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(models)

        model = models.FeedForwardNeuralNetModel(input_dim, hidden_dim, output_dim)
        model.to(self.device)
        model = model.double()
        model.__dict__['_modules'] = load(open(self.model_filepath, 'rb')) 
        self.model = model
        self.ready = True
    
    def predict(self, context, data, confidence=0.8):
        s_data = json.dumps(data)
        context.logger.info("s_data: {}".format(s_data))
        # prepares image for use
        encoded = json.loads(s_data)['content']
        image = jpg_str_to_frame(encoded)
        
        # converts image format to RGB for comptability with face_recognition library and resize for faster processing
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(image, width=750)
        ratio = image.shape[1] / float(rgb.shape[1])
        
        #gets mapping from label to name and known encodings
        classes = self.convert_classes_path_for_pandas()
        idx2name_df = pd.read_csv(classes).set_index('value')
        self.n_classes = len(idx2name_df)
        
        if not self.model:
            self.load()
        
        #locates faces in image and extracts embbeding vector for each face
        context.logger.info('recognizing faces')
        boxes = face_recognition.face_locations(rgb, model='hog')
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        #determines if face is a clear match/ambiguous.
        names = []
        labels = []
        confidences = []
        for encoding in encodings:
            name = 'unknown'
            label = torch.tensor(-1)
            enc_tensor = torch.tensor(encoding, device=self.device)
            self.model.to(self.device)
            out = self.model(enc_tensor)
            pred_n, pred_i = out.topk(1)
            distrib = F.softmax(out, dim=0)
            
            max_p, max_i = distrib.topk(1)
            if max_p.item() > confidence:
                label = pred_i.cpu()
                name = idx2name_df.loc[label]['name'].values[0].replace('_', ' ')
            names.append(name)
            labels.append(label)
            confidences.append(max_p.item())

        #draw boxes with name on the image and performs logic according to match/ambiguous 
        resp_list = []
        for ((top, right, bottom, left), name, encoding, label, confidence) in zip(boxes, names, encodings, labels, confidences):  

            #rescale the face coordinates
            top = int(top * ratio)
            right = int(right * ratio)
            bottom = int(bottom * ratio)
            left = int(left * ratio)
            
            #appends box name and confidence to the returned list  
            resp_list.append({'coords': (top, right, bottom, left), 'name': name, 'label': label.item(), 'confidence': confidence, 'encoding': encoding.tolist()})
            
        return context.Response(body=json.dumps(resp_list))

### Main function
simply initializes the model class and invokes the predict method 

In [105]:
model = PytorchModel()
def handler(context, event):
    return model.predict(context=context, data=event.body)

In [106]:
#nuclio: end-code

### Set configuration for deployment

In [107]:
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io
fn = code_to_function('nuclio-face-prediction', kind='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).apply(mount_v3io())

# set environment variables
fn.set_env('MODELS_PATH',models_path)
fn.set_env('MODEL_PATH', model_path)
fn.set_env('CLASSES_MAP', classes_map)
fn.set_env('V3IO_ACCESS_KEY', os.environ['V3IO_ACCESS_KEY'])

### Deploy the function to the cluster
May take a few minutes due to building of image

In [108]:
from os import getenv, path
PROJECT_BASE_NAME = "faces"
project_name = '-'.join(filter(None, [PROJECT_BASE_NAME, getenv('V3IO_USERNAME', None)]))
addr = fn.deploy(project=project_name)

> 2020-12-14 11:07:04,020 [info] Starting remote function deploy
2020-12-14 11:07:04  (info) Deploying function
2020-12-14 11:07:04  (info) Building
2020-12-14 11:07:04  (info) Staging files and preparing base images
2020-12-14 11:07:04  (info) Building processor image
2020-12-14 11:07:05  (info) Build complete
2020-12-14 11:07:15  (info) Function deploy complete
> 2020-12-14 11:07:15,940 [info] function deployed, address=default-tenant.app.app-lab-eks-b84.iguazio-cd1.com:31869


In [71]:
#!pip install nuclio_sdk

In [109]:

import nuclio_sdk
import logging

os.environ['MODEL_PATH']=model_path
os.environ['CLASSES_MAP']=classes_map
os.environ['MODELS_PATH']=models_path
os.environ['DATA_PATH']=data_path

logger = logging.Logger('triggering')
ctx = nuclio_sdk.Context(logger=logger)

payload = {'file_path': 'pp.jpeg', 'time': '20191110131130', 'camera': 'cammy'}
ev = nuclio_sdk.Event(body=payload)
handler(ctx,ev)

KeyError: 'content'